<img src="../common/fun_long_logo-01.png">

# Tutorial on how to automatically generate reactions using ChemicalFun and write them to phreeqc format
Miron et al., (2023). ThermoFun: A C++/Python library for computing standard thermodynamic properties of substances and reactions across wide ranges of temperatures and pressures. Journal of Open Source Software, 8(83), 4624

[![DOI](https://joss.theoj.org/papers/10.21105/joss.04624/status.svg)](https://doi.org/10.21105/joss.04624)

### Import ThermoFun module
ThermoFun delivers thermodynamic properties of substances and reactions at the temperature and pressure of interest.

In [1]:
import thermofun as fun

### Initialize a ThermoFun database and an engine class

In [2]:
# initalize database using a local file
REE = fun.Database('gemsfiles/HREE-P-OH_3K-fun.json')

In [3]:
print(REE.mapElements().keys())

dict_keys(['Ce', 'Cl', 'Dy', 'Er', 'Eu', 'Gd', 'H', 'Ho', 'La', 'Lu', 'Na', 'Nd', 'O', 'P', 'Pr', 'Sm', 'Tb', 'Tm', 'Y', 'Yb', 'Zz'])


In [14]:
# use the fileter 
from common.filters import substances_containing_elements
# what substances are avaialble containing elements
substances_containing_elements('gemsfiles/HREE-P-OH_3K-fun.json', ['Ce', 'Dy', 'Eu', 'Gd', 'Ho'], False)

symbol               formula                                  class               
Ce(H2PO4)+2          CeH2PO4+2                                SC_AQSOLUTE         
Ce+3                 Ce+3                                     SC_AQSOLUTE         
CeCl+2               CeCl+2                                   SC_AQSOLUTE         
CeCl2+               CeCl2+                                   SC_AQSOLUTE         
CeCl3@               CeCl3@                                   SC_AQSOLUTE         
CeO+                 CeO+                                     SC_AQSOLUTE         
CeO2-                CeO2-                                    SC_AQSOLUTE         
CeO2H@               CeO2H@                                   SC_AQSOLUTE         
CeOH+2               CeOH+2                                   SC_AQSOLUTE         
Ce+4                 Ce|4|+4                                  SC_AQSOLUTE         
Dy(H2PO4)+2          DyH2PO4+2                                SC_AQSOLUTE         
Dy+3

### generate reactions for selected substances

In [6]:
# create a list of symbols of reaction dependent substance
substances_symbols = ["CePO4", "DyPO4", "EuPO4", "GdPO4", "HoPO4", "LaPO4", "LuPO4", "NdPO4", "PrPO4", "SmPO4", "TbPO4", "TmPO4", "YPO4", "YbPO4",
                     "rhabdophane-(Ce)", "rhabdophane-(Gd)", "rhabdophane-(La)", "rhabdophane-(Nd)", "rhabdophane-(Pr)", "rhabdophane-(Sm)"]

### extract them from the database 

In [8]:
substances = [REE.getSubstance(s) for s in substances_symbols]

In [9]:
# set the list of master species
aqueous_master_species = ['H2O@','PO4-3', 'Ce+3', 'Dy+3', 'Eu+3', 'Gd+3', 'Ho+3', 'La+3', 'Lu+3', 'Nd+3', 'Pr+3', 'Sm+3', 'Tb+3', 'Tm+3', 'Y+3', 'Yb+3']

### ChamicalFun library contains functions to automatically generate reactions for given dependent substance and master species 

In [10]:
# import chamicalfun library
import chemicalfun as cfun

### Reactions generator

In [11]:
# using chemicalfun library to generate chemical reactions based on list of master species
substances_formulas = aqueous_master_species+[s.formula() for s in substances]
substances_symbols = aqueous_master_species+[s.symbol() for s in substances]

chemicalReactions = cfun.ChemicalReactions(substances_formulas,substances_symbols)

reactions = chemicalReactions.generateReactions(formation=True) # returns the reactions list as a list of tuples ('substance', coefficient)

# list of dictionaries, with reaction substances as keys and the reaction coefficients as values
reactions_dic = [{el[0]: el[1] for el in r} for r in reactions] 

# strings of reactions that can be used in ThermoFun to calculate the logK at different T and P
reactions_list = chemicalReactions.stringReactions()

for r in reactions_list:
    print(r)
#reactions_dic

Dy+3 + PO4-3 = DyPO4
Eu+3 + PO4-3 = EuPO4
Gd+3 + PO4-3 = GdPO4
Ho+3 + PO4-3 = HoPO4
La+3 + PO4-3 = LaPO4
Lu+3 + PO4-3 = LuPO4
Nd+3 + PO4-3 = NdPO4
Pr+3 + PO4-3 = PrPO4
Sm+3 + PO4-3 = SmPO4
Tb+3 + PO4-3 = TbPO4
Tm+3 + PO4-3 = TmPO4
Y+3 + PO4-3 = YPO4
Yb+3 + PO4-3 = YbPO4
Ce+3 + PO4-3 + 0.732H2O@ = rhabdophane-(Ce)
Gd+3 + PO4-3 + 0.533H2O@ = rhabdophane-(Gd)
La+3 + PO4-3 + 0.804H2O@ = rhabdophane-(La)
Nd+3 + PO4-3 + 0.746H2O@ = rhabdophane-(Nd)
Ce+3 + PO4-3 = CePO4
Sm+3 + PO4-3 + 0.636H2O@ = rhabdophane-(Sm)


In [12]:
from common.export import to_phreeqc
engine = fun.ThermoEngine(REE)

In [13]:
to_phreeqc('phreeqc_data.txt', engine, substances, reactions_list, reactions_dic, True)

/home/dmiron/miniconda3/envs/thermofun-jupyter/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
